#### A 3 node experiment to demonstrate the network simulation and attack process in cyborg

In [36]:
import random
from os.path import dirname
from pprint import pprint
import inspect
from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator
from CybORG.Simulator.Actions import *
from CybORG.Agents import RandomAgent
import numpy as np
import time

path = str(inspect.getfile(CybORG))
#path= dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario_3nodes.yaml'
path = f'C:/Users/HPP/Desktop/cyborg/vu_cyborg/CybORG/CybORG/Simulator/Scenarios/scenario_files/Scenario_rg1.yaml'
sg = FileReaderScenarioGenerator(path)
print('Scenario file path is:',path)

env = CybORG(scenario_generator=sg)
#pprint(dir(env))

#pprint('IP map of network is:',env.get_ip_map())
#pprint('Red agent state is:',env.get_agent_state('Red'))
#pprint('Red agent action space is:',env.get_action_space('Red'))
print('==> IP map of network is:')
pprint(env.get_ip_map())



ips= env.get_ip_map()
userip= ips['User0']
print('-> user ip to attack is:',userip)





results = env.reset(agent='Red')

Scenario file path is: C:/Users/HPP/Desktop/cyborg/vu_cyborg/CybORG/CybORG/Simulator/Scenarios/scenario_files/Scenario_rg1.yaml
==> IP map of network is:
{'Defender': IPv4Address('10.0.244.92'),
 'User0': IPv4Address('10.0.244.87'),
 'User_router': IPv4Address('10.0.244.94')}
-> user ip to attack is: 10.0.244.87


##### Observed state from the perspective of red agent

In [12]:
print('==> Red Agents observed state  is:')
pprint(env.get_agent_state('Red'))

==> Red Agents observed state  is:
{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.0.41'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.0.32/28')}],
           'Processes': [{'PID': 3376, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 0,
                         'PID': 3376,
                         'Timeout': 0,
                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                         'Username': 'SYSTEM'}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User0',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.WINDOWS: 2>,
                           'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>,
                           'position': array([0., 0.])}},
 'success': <TrinaryEnum

In [13]:
print('==> Blue Agents observed state  is:')
pprint(env.get_agent_state('Blue'))

==> Blue Agents observed state  is:
{'Defender': {'Interface': [{'IP Address': IPv4Address('10.0.0.46'),
                             'Interface Name': 'eth0',
                             'Subnet': IPv4Network('10.0.0.32/28')}],
              'Processes': [{'PID': 2296, 'Username': 'ubuntu'},
                            {'PID': 2298, 'Username': 'ubuntu'}],
              'Sessions': [{'Agent': 'Blue',
                            'ID': 0,
                            'PID': 2296,
                            'Timeout': 0,
                            'Type': <SessionType.VELOCIRAPTOR_SERVER: 8>,
                            'Username': 'ubuntu'},
                           {'Agent': 'Blue',
                            'ID': 2,
                            'PID': 2298,
                            'Timeout': 0,
                            'Type': <SessionType.VELOCIRAPTOR_CLIENT: 7>,
                            'Username': 'ubuntu'}],
              'System info': {'Architecture': <Architectur

#### Action space to the red agent:

In [27]:
print('==> Red Agent action space is:')
pprint(env.get_action_space('Red'))
red_actions= env.get_action_space('Red')["action"]
print(red_actions)

==> Red Agent action space is:
{'action': {<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True},
 'agent': {'Red': True},
 'hostname': {'Defender': False, 'User0': True, 'User_router': False},
 'ip_address': {IPv4Address('10.0.32.194'): True,
                IPv4Address('10.0.32.203'): False,
                IPv4Address('10.0.32.205'): False},
 'password': {'vagrant': False},
 'port': {21: False, 22: False, 53: False},
 'process': {1: False,
             389: False,
             407: False,
             409: False,
             560: False,
             656: False,
             790: False,
             802: False,
             803: False,
             807: False,
             824: False,
             825: False,
             827: False,
             832: Fals

In [17]:
print('==> Blue Agent action space is:')
pprint(env.get_action_space('Blue'))

==> Blue Agent action space is:
{'action': {<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.Remove.Remove'>: True,
            <class 'CybORG.Simulator.Actions.AbstractActions.Analyse.Analyse'>: True},
 'agent': {'Blue': True},
 'hostname': {'Defender': True, 'User0': True, 'User_router': False},
 'ip_address': {IPv4Address('10.0.0.35'): False,
                IPv4Address('10.0.0.41'): True,
                IPv4Address('10.0.0.46'): True},
 'password': {'vagrant': True},
 'port': {21: False, 22: False, 53: False},
 'process': {1: False,
             389: False,
             407: False,
             409: False,
             560: False,
             656: False,
             790: False,
             802: False,
             803: False,
             807: False,
             824: False,
             825: False,
             827: False,
             832: False,
             844: False,
             847: False,
             8

#### Attack action : DiscoverNetworkServices
It allow the red agent to identify services on a host as a prerequisite for running an exploit.
It Calls the low level action 'PortScan' then modifies the observation. The outcome is all open ports running on the host.  It must be used on a host to successfully run the high level action ExploitRemoteServices. 


In [4]:
#creating and executing port discovery attack on user network 
action = DiscoverNetworkServices(session=0,agent='Red',ip_address=userip)
results = env.step(action=action,agent='Red')
print('==> Red agent state after discover network services attack:')
pprint(results.observation)



==> Red agent state after discover network services attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216')}],
                  'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22}]},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 21}]}]},
 'success': <TrinaryEnum.TRUE: 1>}


In [18]:
#creating and executing port discovery attack on user network 
action_space = env.get_action_space('Red')
#pprint(action_space)
subnets=action_space["subnet"]
known_subnets = [subnet for subnet in subnets if subnets[subnet]]
subnet = known_subnets[0]

print('Subnet visible to red is:',subnet)
action = DiscoverRemoteSystems(subnet = subnet, session=0,agent='Red')
results = env.step(action=action,agent='Red')
print('==> Red agent state after discover network services attack:')
pprint(results.observation)

Subnet visible to red is: 10.0.173.208/28
==> Red agent state after discover network services attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216'),
                                 'Subnet': IPv4Network('10.0.173.208/28')}]},
 '10.0.173.222': {'Interface': [{'IP Address': IPv4Address('10.0.173.222'),
                                 'Subnet': IPv4Network('10.0.173.208/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


#### Attack action : ExploitRemoteServices
It allow the red agent to examines the target host and returns a selected applicable escalate action if any, as well as processes that are required to be genuine.

In [19]:
#creating and executing attack on user network (can we attack on router? -dont know)
action = ExploitRemoteService(ip_address=userip,session=0,agent='Red')
results = env.step(action=action,agent='Red')
print('==> Red agent state after exploit attack:')
pprint(results.observation)

==> Red agent state after exploit attack:
{'10.0.173.216': {'Interface': [{'IP Address': IPv4Address('10.0.173.216')}],
                  'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                  'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22}],
                                 'Process Type': <ProcessType.SSH: 2>},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 51418,
                                                  'remote_address': IPv4Address('10.0.173.216'),
                                                  'remote_port': 22}]},
                                {'Connections': [{'local_address': IPv4Address('10.0.173.216'),
                                                  'local_port': 22,
                                                  'remote_

In [62]:
red_actions=['Sleep','DiscoverNetworkServices','ExploitRemoteService']
blue_actions=['Sleep','Analyse','Remove']
states=['blue','green','orange','red']
initial_state='blue'
log=[] #log [init_state,red_Action,state,blue_action,state,....] 
log.append(initial_state)

In [63]:
steps=10
random_actions=np.random.randint(3,size=steps*2)
print('Ranom actions are:',random_actions)
for i in range(0,len(random_actions),2):
    #first red random action
    red_action_id=random_actions[i]
    red_action=red_actions[red_action_id]
    print('Red agent action is:',red_action)
    log.append(red_action)
    #results = env.step(action=action, agent='Red')
    
    #todo: log state of node here
    
    
    #second blue random action
    blue_action_id=random_actions[i+1]
    blue_action= blue_actions[blue_action_id]
    print('Blue agent action is:',blue_action)
    log.append(blue_action)  
   
    
    #results = env.step(action=action, agent='Red')
     #todo: log state of node here
print('log is:',log)    

Ranom actions are: [2 1 2 1 2 0 1 2 0 2 2 0 0 2 0 0 0 0 2 2]
Red agent action is: ExploitRemoteService
Blue agent action is: Analyse
Red agent action is: ExploitRemoteService
Blue agent action is: Analyse
Red agent action is: ExploitRemoteService
Blue agent action is: Sleep
Red agent action is: DiscoverNetworkServices
Blue agent action is: Remove
Red agent action is: Sleep
Blue agent action is: Remove
Red agent action is: ExploitRemoteService
Blue agent action is: Sleep
Red agent action is: Sleep
Blue agent action is: Remove
Red agent action is: Sleep
Blue agent action is: Sleep
Red agent action is: Sleep
Blue agent action is: Sleep
Red agent action is: ExploitRemoteService
Blue agent action is: Remove
log is: ['blue', 'ExploitRemoteService', 'Analyse', 'ExploitRemoteService', 'Analyse', 'ExploitRemoteService', 'Sleep', 'DiscoverNetworkServices', 'Remove', 'Sleep', 'Remove', 'ExploitRemoteService', 'Sleep', 'Sleep', 'Remove', 'Sleep', 'Sleep', 'Sleep', 'Sleep', 'ExploitRemoteService', 